## Pre-processing des data

1) Data cleaning

a) FBREF

In [18]:
import numpy as np
import pandas as pd
import pickle
import zlib

In [ ]:
glossary = pd.read_csv("fbref_data\\Glossaire_FBREF.csv")
glossary.head()

,Unnamed: 0,0
0,Season,Season
1,Age,Age at season start
2,Country,Flag and name of the country.
3,Comp,Competition
4,LgRank,Squad finish in competition


In [ ]:
glossary.describe()

glossary.rename(columns={"Unnamed: 0" : "Name", "0" : "Description"}, inplace = True)
glossary.head()

,Name,Description
0,Season,Season
1,Age,Age at season start
2,Country,Flag and name of the country.
3,Comp,Competition
4,LgRank,Squad finish in competition


In [ ]:
player_stats_fbref=pickle.loads(zlib.decompress(open('fbref_data/compressed_player_stats_fbref_1.bin','rb').read()))
player_stats_fbref.update(pickle.loads(zlib.decompress(open('fbref_data/compressed_player_stats_fbref_2.bin','rb').read())))
player_stats_fbref=pd.DataFrame(player_stats_fbref).transpose()
player_stats_fbref=player_stats_fbref.sort_index(axis=1,ascending=False)

teams_fbref=pickle.load(open('fbref_data/teams_fbref.pkl','rb'))

In [20]:
player_stats_fbref.head()

,meta,2024-2025,2024,2023-2024,2023,2022-2023,2022,2021-2022,2021,2020-2021,...,1999-2000,1999,1998-1999,1998,1997-1998,1996-1997,1995-1996,1994-1995,1993-1994,1992-1993
César Fernández,"{'height': '', 'weight': '', 'preferred foot':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yoel Lago,"{'height': '', 'weight': '', 'preferred foot':...","{'age': 20.0, 'team': 'Celta Vigo', 'country':...",NaN,"{'age': 19.0, 'team': 'Celta Vigo', 'country':...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Marc Vidal,"{'height': '', 'weight': '', 'preferred foot':...","{'age': 24.0, 'team': 'Celta Vigo', 'country':...",NaN,NaN,NaN,"{'age': 22.0, 'team': 'FC Andorra', 'country':...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rui Silva,"{'height': 189.0, 'weight': 84.0, 'preferred f...","{'age': 30.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 29.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 28.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 27.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 26.0, 'team': 'Granada', 'country': 'e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abde Ezzalzouli,"{'height': 177.0, 'weight': 73.0, 'preferred f...","{'age': 22.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 21.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 20.0, 'team': 'Osasuna', 'country': 'e...",NaN,"{'age': 19.0, 'team': 'Barcelona', 'country': ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


b)Sofascore

In [15]:
sf_data = pd.read_csv("sf_data\player_ratings.csv")
sf_data.head()

,Unnamed: 0,2024-2025,2023-2024,2022-2023,2021-2022,2020-2021,2019-2020,2018-2019,2017-2018,2016-2017,2015-2016
0,Bukayo Saka,8.04,7.36,5.97,7.67,6.43,6.04,6.25,NaN,NaN,NaN
1,Neto,9.41,7.31,4.00,7.71,7.09,6.52,8.29,7.84,8.20,8.0
2,Declan Rice,8.63,7.98,5.95,8.14,6.30,7.03,7.95,7.26,NaN,NaN
3,Gabriel Magalhães,6.70,7.86,6.00,6.66,5.56,6.68,6.00,7.12,5.89,NaN
4,Martin Ødegaard,6.95,7.14,8.25,8.36,5.79,6.51,NaN,NaN,NaN,NaN
